In [1]:
import livef1
from livef1.utils.constants import interpolation_map

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

session = livef1.get_session(season=2024, meeting_identifier="Spa", session_identifier="Race")

2025-01-13 13:46:17,002 : livef1 : INFO :: Selected meeting/session is:
Meeting Offname : FORMULA 1 ROLEX BELGIAN GRAND PRIX 2024
Meeting Name : Belgian Grand Prix
Meeting Circuit Shortname : Spa-Francorchamps
2025-01-13 13:46:17,004 : livef1 : INFO :: The meeting was received successfully.
2025-01-13 13:46:17,009 : livef1 : INFO :: Selected meeting/session is:
Session Name : Race
2025-01-13 13:46:17,010 : livef1 : INFO :: The session was received successfully.


In [2]:
res = session.get_data(dataName="Timing_Data")
df_exp = pd.DataFrame(res.value)
df_exp

2025-01-13 13:46:17,655 : livef1 : INFO :: Getting requested data.
Selected session : 2024 Belgian Grand Prix Race
Topic : TimingData
2025-01-13 13:46:20,266 : livef1 : INFO :: Data is successfully received.
2025-01-13 13:46:20,429 : livef1 : INFO :: Data is successfully parsed.


,SessionKey,timestamp,DriverNo,GapToLeader,IntervalToPositionAhead_Value,IntervalToPositionAhead_Catching,Line,Position,ShowPosition,RacingNumber,...,Sectors_2_Segments_6_Status,Sectors_1_PreviousValue,NumberOfLaps,Sectors_2_PreviousValue,Sectors_0_Value,Sectors_0_OverallFastest,Sectors_0_PersonalFastest,Sectors_0_PreviousValue,BestLapTime_Lap,NumberOfPitStops
0,9574,00:00:03.734,16,,,False,1.0,1,True,16,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,9574,00:00:03.734,11,,,False,2.0,2,True,11,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,9574,00:00:03.734,44,,,False,3.0,3,True,44,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,9574,00:00:03.734,4,,,False,4.0,4,True,4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,9574,00:00:03.734,81,,,False,5.0,5,True,81,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47650,9574,02:17:07.716,1,NaN,NaN,True,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
47651,9574,02:17:07.716,10,NaN,NaN,True,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
47652,9574,02:17:07.716,77,NaN,NaN,True,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
47653,9574,02:17:07.716,20,NaN,NaN,True,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
df_11 = df_exp[df_exp["DriverNo"] == "11"]

In [6]:
df_11.columns.tolist()

['SessionKey',
 'timestamp',
 'DriverNo',
 'GapToLeader',
 'IntervalToPositionAhead_Value',
 'IntervalToPositionAhead_Catching',
 'Line',
 'Position',
 'ShowPosition',
 'RacingNumber',
 'Retired',
 'InPit',
 'PitOut',
 'Stopped',
 'Status',
 'Sectors_1_Stopped',
 'Sectors_1_Value',
 'Sectors_1_Status',
 'Sectors_1_OverallFastest',
 'Sectors_1_PersonalFastest',
 'Sectors_2_Stopped',
 'Sectors_2_Value',
 'Sectors_2_Status',
 'Sectors_2_OverallFastest',
 'Sectors_2_PersonalFastest',
 'Sectors_3_Stopped',
 'Sectors_3_Value',
 'Sectors_3_Status',
 'Sectors_3_OverallFastest',
 'Sectors_3_PersonalFastest',
 'Speeds_I1_Value',
 'Speeds_I1_Status',
 'Speeds_I1_OverallFastest',
 'Speeds_I1_PersonalFastest',
 'Speeds_I2_Value',
 'Speeds_I2_Status',
 'Speeds_I2_OverallFastest',
 'Speeds_I2_PersonalFastest',
 'Speeds_FL_Value',
 'Speeds_FL_Status',
 'Speeds_FL_OverallFastest',
 'Speeds_FL_PersonalFastest',
 'Speeds_ST_Value',
 'Speeds_ST_Status',
 'Speeds_ST_OverallFastest',
 'Speeds_ST_PersonalFas

In [63]:
from datetime import timedelta

seconds = (timedelta(seconds=31.365) + timedelta(seconds=49.207) + timedelta(seconds=29.668)).total_seconds()
(seconds % 3600) // 60, seconds % 60


(1.0, 50.239999999999995)

In [161]:
cols = ["LastLapTime_Value","Sectors_0_Value","Sectors_1_Value","Sectors_2_Value","Sectors_3_Value","Sectors_0_PreviousValue","Sectors_1_PreviousValue","Sectors_2_PreviousValue","NumberOfLaps"]

def str_timedelta(x):
    if isinstance(x,str):
        count_sep = x.count(":")

        if count_sep == 0:
            return "00:00:" + x
        elif count_sep == 1:
            return "00:" + x
        else: return x
    else:
        return x


df_test = df_11[["timestamp"] + cols ].dropna(subset=cols, how="all").replace('',np.nan)

# df_test["LapElapsed"] = df_test["Sectors_0_Value"] + df_test["Sectors_1_Value"] + df_test["Sectors_2_Value"]
# for col in df_test.columns:
#     df_test[col] = pd.to_timedelta(df_test[col])


for col in ["Sectors_0_Value","Sectors_1_Value","Sectors_2_Value","Sectors_3_Value"]:
    df_test[col] = df_test[col]
    # df_test[col] = df_test[col].fillna("00:00:00.000")
    df_test[col] = pd.to_timedelta(df_test[col].apply(str_timedelta))

for col in ["Sectors_0_PreviousValue","Sectors_1_PreviousValue","Sectors_2_PreviousValue"]:
    df_test[col] = df_test[col]
    # df_test[col] = df_test[col].fillna("00:00:00.000")
    df_test[col] = pd.to_timedelta(df_test[col].apply(str_timedelta))


df_test["LapElapsed"] = df_test["Sectors_0_Value"] + df_test["Sectors_1_Value"] + df_test["Sectors_2_Value"]


df_test.iloc[150:200]
df_test.tail(50)

C:\Users\EU10698\AppData\Local\Temp\ipykernel_25744\3794797740.py:16: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_test = df_11[["timestamp"] + cols ].dropna(subset=cols, how="all").replace('',np.nan)


,timestamp,LastLapTime_Value,Sectors_0_Value,Sectors_1_Value,Sectors_2_Value,Sectors_3_Value,Sectors_0_PreviousValue,Sectors_1_PreviousValue,Sectors_2_PreviousValue,NumberOfLaps,LapElapsed
38258,01:59:25.543,1:47.843,NaT,NaT,0 days 00:00:29.177000,NaT,NaT,NaT,NaT,35.0,NaT
38284,01:59:27.552,NaN,NaT,NaT,NaT,NaT,NaT,NaT,0 days 00:00:29.177000,NaN,NaT
38566,01:59:56.726,NaN,0 days 00:00:31.205000,NaT,NaT,NaT,NaT,NaT,NaT,NaN,NaT
38569,01:59:56.975,NaN,NaT,NaT,NaT,NaT,0 days 00:00:31.205000,NaT,NaT,NaN,NaT
39017,02:00:44.348,NaN,NaT,0 days 00:00:47.674000,NaT,NaT,NaT,NaT,NaT,NaN,NaT
39045,02:00:47.663,NaN,NaT,NaT,NaT,NaT,NaT,0 days 00:00:47.674000,NaT,NaN,NaT
39266,02:01:13.482,1:48.014,NaT,NaT,0 days 00:00:29.135000,NaT,NaT,NaT,NaT,36.0,NaT
39305,02:01:17.694,NaN,NaT,NaT,NaT,NaT,NaT,NaT,0 days 00:00:29.135000,NaN,NaT
39578,02:01:44.837,NaN,0 days 00:00:31.170000,NaT,NaT,NaT,NaT,NaT,NaT,NaN,NaT
39604,02:01:47.702,NaN,NaT,NaT,NaT,NaT,0 days 00:00:31.170000,NaT,NaT,NaN,NaT


In [133]:
cols = ["Sectors_2_Value","Sectors_2_PreviousValue"]
df_test = df_11[["timestamp"] + cols].replace("",np.nan).dropna(subset=cols, how="all")
df_test["timestamp"] = pd.to_timedelta(df_test["timestamp"])

df_test["diff"] = df_test["timestamp"] - df_test["timestamp"].shift(1)
df_test
# df_test.dropna(subset="Sectors_2_PreviousValue")

,timestamp,Sectors_2_Value,Sectors_2_PreviousValue,diff
3199,0 days 00:57:08.631000,29.491,NaN,NaT
3215,0 days 00:57:10.473000,NaN,29.491,0 days 00:00:01.842000
4321,0 days 00:58:58.806000,29.668,NaN,0 days 00:01:48.333000
4325,0 days 00:58:59.549000,NaN,29.668,0 days 00:00:00.743000
5373,0 days 01:00:48.902000,29.600,NaN,0 days 00:01:49.353000
...,...,...,...,...
45395,0 days 02:12:07.298000,34.064,NaN,0 days 00:01:49.045000
45405,0 days 02:12:08.294000,NaN,34.064,0 days 00:00:00.996000
46572,0 days 02:14:10.678000,30.034,NaN,0 days 00:02:02.384000
46599,0 days 02:14:13.574000,NaN,30.034,0 days 00:00:02.896000


In [27]:
df_11["Speeds_I1_Value"].unique()

array(['', nan, '316', '325', '314', '320', '312', '321', '315', '310',
       '329', '322', '328', '319', '313', '323', '317', '306', '307',
       '309', '311', '324', '303'], dtype=object)

In [29]:
df_11

,SessionKey,timestamp,DriverNo,GapToLeader,IntervalToPositionAhead_Value,IntervalToPositionAhead_Catching,Line,Position,ShowPosition,RacingNumber,...,Sectors_2_Segments_6_Status,Sectors_1_PreviousValue,NumberOfLaps,Sectors_2_PreviousValue,Sectors_0_Value,Sectors_0_OverallFastest,Sectors_0_PersonalFastest,Sectors_0_PreviousValue,BestLapTime_Lap,NumberOfPitStops
1,9574,00:00:03.734,11,,,False,2.0,2,True,11,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21,9574,00:00:05.474,11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
57,9574,00:11:50.163,11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
66,9574,00:11:57.383,11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
73,9574,00:12:02.357,11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47461,9574,02:15:55.478,11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
47478,9574,02:15:58.766,11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,28.584,NaN,NaN,NaN,NaN,NaN,NaN
47485,9574,02:15:59.906,11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
47636,9574,02:16:49.662,11,NaN,NaN,False,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
def build_timing_table(df):

    timing_df = df.copy()

    timing_df["InPit"] = timing_df["InPit"].fillna(False).astype(int).cumsum()
    return timing_df[["timestamp","NumberOfLaps","NumberOfPitStops","InPit"]]

build_timing_table(df_11)

C:\Users\EU10698\AppData\Local\Temp\ipykernel_25744\2576228234.py:5: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  timing_df["InPit"] = timing_df["InPit"].fillna(False).astype(int).cumsum()


,timestamp,NumberOfLaps,NumberOfPitStops,InPit
1,00:00:03.734,NaN,NaN,1
21,00:00:05.474,NaN,NaN,1
57,00:11:50.163,NaN,NaN,1
66,00:11:57.383,NaN,NaN,1
73,00:12:02.357,NaN,NaN,1
...,...,...,...,...
47461,02:15:55.478,NaN,NaN,7
47478,02:15:58.766,NaN,NaN,7
47485,02:15:59.906,NaN,NaN,7
47636,02:16:49.662,NaN,NaN,7


In [34]:
df_11[df_11["InPit"] == True]

,SessionKey,timestamp,DriverNo,GapToLeader,IntervalToPositionAhead_Value,IntervalToPositionAhead_Catching,Line,Position,ShowPosition,RacingNumber,...,Sectors_2_Segments_6_Status,Sectors_1_PreviousValue,NumberOfLaps,Sectors_2_PreviousValue,Sectors_0_Value,Sectors_0_OverallFastest,Sectors_0_PersonalFastest,Sectors_0_PreviousValue,BestLapTime_Lap,NumberOfPitStops
1,9574,00:00:03.734,11,,,False,2.0,2,True,11,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
385,9574,00:13:42.506,11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
775,9574,00:15:51.157,11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1236,9574,00:18:02.529,11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13605,9574,01:15:29.327,11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
24002,9574,01:33:57.247,11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0
45353,9574,02:12:03.159,11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0
